# Setting up

In [4]:
import pandas as pd 
import numpy as np
import sklearn

In [5]:
interaction_2020_path = "../uwwi_datasets/uwwi_dataset_interactions/uwwi_dataset_interactions_2020.csv"
interaction_2021_path = "../uwwi_datasets/uwwi_dataset_interactions/uwwi_dataset_interactions_2021.csv"
interaction_2022_path = "../uwwi_datasets/uwwi_dataset_interactions/uwwi_dataset_interactions_2022.csv"
agencies_path = "../uwwi_datasets/uwwi_dataset_agencies.csv"
clients_path = "../uwwi_datasets/uwwi_dataset_clients_v2.csv"
services_path = "../uwwi_datasets/uwwi_dataset_services.csv"
sites_path = "../uwwi_datasets/uwwi_dataset_sites.csv"

In [6]:
import chardet
def get_encoding(file_path):

    with open(file_path,'rb') as f:
            data = f.read(10000)
    return chardet.detect(data).get("encoding")

# Getting into Clients

In [129]:
file_path = clients_path
df = pd.read_csv(file_path, sep=',')
# df.head(5)

C:\Users\Siddiquee\AppData\Local\Temp\ipykernel_5156\595301242.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=',')


In [130]:
df = df[(df.ClientAddressus_ClientAddressus_state.notna()) & (df.ClientAddressus_ClientAddressus_county.notna())]
df.ClientAddressus_ClientAddressus_state = df.ClientAddressus_ClientAddressus_state.str.lower().str.strip().astype("string")
df.ClientAddressus_ClientAddressus_county = df.ClientAddressus_ClientAddressus_county.str.lower().str.strip().astype("string")

In [89]:
counties = df[df.ClientAddressus_ClientAddressus_state.str.lower() == 'wi'].ClientAddressus_ClientAddressus_county.unique().tolist()
counties = list(set([name.strip() for name in counties]))
counties.sort()
# with open('./county_names_clients.txt', 'w') as file:
#     for line in counties:
#         file.write(f"{line}\n")
# df.ClientAddressus_ClientAddressus_county[df.ClientAddressus_ClientAddressus_state.isna()]

In [101]:
cities = df[(df.ClientAddressus_ClientAddressus_state.str.lower() == 'wi') & (df.ClientAddressus_ClientAddressus_county.str.strip().str.lower() == 'county')].ClientAddressus_ClientAddressus_city.tolist()
cities = list(set([str(city).strip().lower() for city in cities]))
cities

['weston', 'milwaukee']

In [125]:
df = df[(df.ClientAddressus_ClientAddressus_state.str.strip().str.lower() == 'wi') 
    & (df.ClientAddressus_ClientAddressus_county.str.strip().str.lower() == 'wi') 
    & (df.ClientAddressus_ClientAddressus_city.str.strip().str.lower() == 'wausau')]['ClientAddressus_ClientAddressus_county'].replace('wi', 'marathon')



# for state and  county 'wi'
# wausau -> marathon 
# waukesha -> waukesha
# milwaukee -> milwaukee
# miwaukee -> milwaukee
# columbus -> columbia
# racine -> racine
# madison -> dane
# rothschild -> marathon

# # for state and  county 'wi' 
# sun prairie -> dane
# spring green -> sauk
# baldwin -> st. croix
# deerbrook -> langlade 
# onalaska -> la crosse
# milwaukee -> milwaukee
# mcfarland -> dane
# fitchburg -> dane
# madison -> dane

# # for state and  county 'county' 
# weston -> marathon
# milwaukee -> milwaukee

In [128]:
df

407375    marathon
Name: ClientAddressus_ClientAddressus_county, dtype: string

In [29]:
cities = df[(df.ClientAddressus_ClientAddressus_state.str.lower() == 'wv') & (df.ClientAddressus_ClientAddressus_city.notna())].ClientAddressus_ClientAddressus_city.str.lower().unique().tolist()
# cities = list(set([str(name).strip() for name in cities]))
# cities.sort()
cities

['parkersburg',
 'accoville',
 'milton',
 'charleston',
 'huntington',
 'morgantown',
 'williamstown',
 'keyser',
 'buckhannon']

In [93]:
df.shape[0] - df[df.ClientAddressus_ClientAddressus_state.str.lower() == 'wi'].shape[0]

19952

In [94]:
df = df[df.ClientAddressus_ClientAddressus_state.str.lower() == 'wi']

In [23]:
df.ClientAddressus_ClientAddressus_zip[df.ClientAddressus_ClientAddressus_zip.isna()].shape

(68132,)

# ADI Scores

In [141]:
SOCIOECONOMIC_FILE = '../external_datasets/socioeconomic_dataset/socioeconomic_scores_zipcode.csv'
df_socioec_scores = pd.read_csv(SOCIOECONOMIC_FILE)
df_socioec_scores.head(5)

,ZIP_4,GISJOIN,FIPS,ADI_NATRANK,ADI_STATERANK,TYPE
0,530010064,G55011700110002,5.511701e+11,47,4,
1,530010061,G55011700110002,5.511701e+11,47,4,
2,530010063,G55011700110002,5.511701e+11,47,4,
3,530010049,G55011700112001,5.511701e+11,45,3,
4,530010050,G55011700112001,5.511701e+11,45,3,


In [99]:
def clean_adi(df_original):
    
    # Let's create the cleaned dataset
    df_clean = df_original.copy()
    
    # Extract only 5 digits from zipcode
    df_clean['ZIP'] = df_clean['ZIP_4'].astype(str).str[:5]
    
    # Remove zipcodes that don't have a valid decile
    df_clean = df_clean.loc[(df_clean.ADI_STATERANK!='GQ') &
                                         (df_clean.ADI_STATERANK!='PH') &
                                          (df_clean.ADI_STATERANK!='GQ-PH') &
                                        (~df_clean.ADI_STATERANK.isna())
                                         ]
    # Transform state rank to integer
    df_clean['ADI_STATERANK'] = df_clean['ADI_STATERANK'].astype(int)
    
    # IMPORTANT: Assign each zipcode the average rank from their block neighborhoods 
    df_clean = df_clean[['ZIP','ADI_STATERANK']].groupby(['ZIP']).\
                                                                agg({'ADI_STATERANK':'mean'}).reset_index()
    
    
    return df_clean[['ZIP','ADI_STATERANK']]

In [101]:
df = clean_adi(df_socioec_scores)
df.head(5)

,ZIP,ADI_STATERANK
0,53001,3.680292
1,53002,4.310638
2,53003,2.000000
3,53004,2.960118
4,53005,1.497231


In [140]:
# df[['ZIP, COUNT']].groupby(['ZIP']).count()
# ['ADI_STATERANK']
counting = df.groupby(["ZIP"]).count().reset_index()
counting[counting.ADI_STATERANK > 1]

,ZIP,ADI_STATERANK


In [144]:
df_socioec_scores.ADI_NATRANK.unique()

array(['47', '45', '59', '44', '57', '26', '43', '39', '33', '41', '54',
       '19', '24', '29', '25', '36', '30', '13', '31', '38', '35', '23',
       '27', '22', '14', '46', '42', '53', '58', '61', '68', '64', '20',
       '37', '34', '21', '18', '32', '52', '67', '56', '78', '50', '63',
       '62', '69', '16', '51', '17', '9', '66', '40', '48', '49', '72',
       '74', '11', 'GQ', '55', '6', '12', '71', '81', '60', '28', '79',
       '76', '75', '8', '7', '77', '65', '10', '5', '87', '92', '93',
       '70', '83', '89', '96', '88', '86', '94', '73', '15', '82', '85',
       'GQ-PH', '97', '91', '84', '80', '100', '90', 'PH', '98', '95',
       '99', '4', nan, '3'], dtype=object)

# SVI Dataset

In [49]:
census_svi_path = "../external_datasets/Wisconsin_CENSUS_SVI.csv"
county_svi_path = "../external_datasets/Wisconsin_COUNTY_SVI.csv"

In [84]:
svi_county_df = pd.read_csv(county_svi_path, sep=',')
# svi_county_df.head(5)

In [86]:
svi_county_df = svi_county_df[svi_county_df.COUNTY.notna()]
svi_county_df.COUNTY = svi_county_df.COUNTY.astype("string")
svi_county_df.COUNTY = svi_county_df.COUNTY.str.strip().str.lower()
# svi_county_df.COUNTY

In [88]:
counties = list(set([str(name).strip() for name in svi_county_df.COUNTY]))
counties.sort()
with open('./wisconsin_county_names_svi.txt', 'w') as file:
    for line in counties:
        file.write(f"{line}\n")